# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096831


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:44,  3.73s/it]

Rendering models:  10%|█         | 3/30 [00:08<01:14,  2.78s/it]

Rendering models:  13%|█▎        | 4/30 [00:08<00:55,  2.14s/it]

Rendering models:  17%|█▋        | 5/30 [00:09<00:41,  1.66s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:21<00:02,  1.53it/s]

Rendering models:  93%|█████████▎| 28/30 [00:22<00:01,  1.88it/s]

Rendering models:  97%|█████████▋| 29/30 [00:22<00:00,  1.82it/s]

Rendering models: 100%|██████████| 30/30 [00:23<00:00,  1.89it/s]

Jnursssmith5263                       0.002400
ethanth                               0.000734
jnarayanbvg                           0.000746
awright5                              0.002521
deivad                                0.001320
wilderbeast2                          0.029577
whitelaughter                         0.002315
Nauter                                0.007875
JoshMcflurry                          0.748511
bryndismr                             0.001316
not-logged-in-a8038c9112f1e417d868    0.002747
not-logged-in-f65c325548a8d20ddd46    0.085840
Mr_DMan                               0.001467
tekhalaf                              0.001393
not-logged-in-4f72c06e57f9d2f5568f    0.003429
MeepTastic                            0.001945
clairedeu                             0.000820
TRISTANM18                            0.001236
Planetme                              0.005845
not-logged-in-300dca7f45d1a5d1dd58    0.005579
Lavadude                              0.031209
crush202020  

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())